In [ ]:
# Median Household Income
# "B19013_001E"
# Median household income in the past 12 months

# Income Per Capita
# "B19301_001E"
# Per capita income in the past 12 months

# Poverty Count
# "B17001_002E"
# Number of persons whose income in the past 12 months is below the poverty level

# Family Poverty Count
# "B17012_002E"
# Number of families below the poverty level in the past 12 months

# Population
# "B01003_001E"
# Total population

# Median Home Value
# "B25077_001E"
# average value of an owner-occupied home

In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key


In [ ]:
zip_csv = "./Resources/ZipCode.csv"
zip_df = pd.read_csv(zip_csv)
zip_df

In [ ]:
# Make an empty dictionary to hold the dfs
census_output = {}

# Make a for loop
for data_year, suffix in zip([2016, 2017, 2018], ["a", "b", "c"]):
    census_data = Census(api_key, year= data_year).acs5.get(("NAME", "B19013_001E", "B19301_001E", "B17001_002E",
                          "B17012_002E", "B01003_001E", "B25077_001E"), {'for': 'zip code tabulation area:*'})
# Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

# Column Reordering
    census_pd = census_pd.rename(columns={
        "B01003_001E": "Population", 
        "B17001_002E": "Poverty Count", 
        "B17012_002E": "Family Poverty Count",
        "B19013_001E": "Median Household Income", 
        "B19301_001E": "Per Capita Income", 
        "B25077_001E": "Median Home Value",
        "NAME": "Name", "zip code tabulation area": "Zipcode"})

    census_pd["Year"] = data_year

# Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = (100 * census_pd["Poverty Count"].astype(float) / census_pd["Population"].astype(float)).round(2)

# Final DataFrame
    census_pd = census_pd[[
        "Year",
        "Zipcode", 
        "Population", 
        "Poverty Count", 
        "Poverty Rate", 
        "Family Poverty Count", 
        "Median Household Income", 
        "Per Capita Income", 
        "Median Home Value"]]

# Filter Zipcode: limited to Sac County
    census_sac = census_pd[census_pd["Zipcode"].isin([
    "95608", "95610", "95615", "95621", "95624", "95626", "95628", "95630", "95632", "95638", 
    "95639", "95641", "95652", "95655", "95660", "95662", "95670", "95673", "95680", "95683",
    "95690", "95693", "95742", "95757", "95758", "95811", "95814", "95815", "95816", "95817",
    "95818", "95819", "95820", "95821", "95822", "95823", "95824", "95825", "95826", "95827",
    "95828", "95829", "95830", "95831", "95832", "95833", "95834", "95835", "95837", "95838", 
    "95841", "95842", "95843", "95864"])]

# Drop bad data
# (Removed 95652, 95680)
    census_sac = census_sac.dropna(how='any')
    census_sac = census_sac.loc[(census_sac["Median Home Value"] > -666666666)]
    census_sac = census_sac.loc[(census_sac["Median Household Income"] > -666666666)]

    census_output[suffix] = census_sac


# output_dfs = {}
   
census_output["a"]


In [62]:
census_output_final = pd.concat([census_output["a"], census_output["b"], census_output["c"]])

In [63]:
census_output_final

,Year,Zipcode,Population,Poverty Count,Poverty Rate,Family Poverty Count,Median Household Income,Per Capita Income,Median Home Value
13504,2016,95608,60199.0,8680.0,14.42,1392.0,56891.0,36796.0,331500.0
13505,2016,95610,44711.0,6767.0,15.13,1075.0,51271.0,25814.0,248000.0
13508,2016,95615,970.0,197.0,20.31,0.0,66080.0,26284.0,309600.0
13513,2016,95621,41908.0,5809.0,13.86,986.0,52462.0,25123.0,196200.0
13515,2016,95624,64429.0,6571.0,10.20,1310.0,84854.0,32386.0,318900.0
...,...,...,...,...,...,...,...,...,...
31255,2018,95838,39187.0,9989.0,25.49,1847.0,44849.0,17299.0,201900.0
31256,2018,95841,20586.0,3964.0,19.26,885.0,47215.0,25167.0,233300.0
31257,2018,95842,32269.0,7050.0,21.85,1386.0,50412.0,22419.0,222300.0
31258,2018,95843,46709.0,5818.0,12.46,1248.0,78102.0,27488.0,299900.0


In [65]:
census_output_final.to_csv("./Resources/census_output_final.csv", encoding="utf-8", index=False)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# # set width of bar
# barWidth = 0.25
# # set height of bar
# bars1 = [12, 30, 1, 8, 22]
# bars2 = [28, 6, 16, 5, 10]
# bars3 = [29, 3, 24, 25, 17]
# # Set position of bar on X axis
# r1 = np.arange(len(bars1))
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]
# # Make the plot
# plt.bar(r1, bars1, color='#7F6D5F', width=barWidth, edgecolor='white', label='var1')
# plt.bar(r2, bars2, color='#557F2D', width=barWidth, edgecolor='white', label='var2')
# plt.bar(r3, bars3, color='#2D7F5E', width=barWidth, edgecolor='white', label='var3')
# # Look at a multiline plot to mix bar
# plt.plot(['A', 'B', 'C', 'D', 'E'],[29, 3, 24, 25, 17])
# # Add xticks on the middle of the group bars
# plt.xlabel('group', fontweight='bold')
# plt.xticks([r + barWidth for r in range(len(bars1))], ['A', 'B', 'C', 'D', 'E'])
# # Create legend & Show graphic
# plt.legend()
# plt.show()